In [52]:
from rdkit import Chem
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Draw
from rdkit.Chem import Fragments
from ClickReaction import BocRemoval
from ClickReaction import CuAAC
from rdkit.Chem import  Descriptors
from rdkit.Chem.Draw import IPythonConsole
import io
import itertools
from PIL import Image
from ClickReaction import BocRemoval
from tqdm import tqdm
import re
from rdkit.Chem import Lipinski
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen
import json
import csv
import numpy as np

In [53]:
def replace_specific_pattern(text, pattern, replacement):
    return re.sub(pattern, replacement, text)
rxn = Chem.ReactionFromSmarts('[*:6][C:1]#[C:2].[N-:3]=[N+:4]=[N:5]>>[N:5]1([C:2]=[C:1]([*:6])[N:3]=[N:4]1)')

In [54]:
def convert_primary_amines_to_diazonium(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    primary_amine_pattern = Chem.MolFromSmarts('[NX3;H2]')  
    diazonium_pattern = Chem.MolFromSmiles('[N]=[N+]=[N-]')
    wrong_smiles = Chem.MolFromSmiles('C([N]=[N+]=[N-])[N]=[N+]=[N-]')
    wrong_smiles_2 = Chem.MolFromSmiles('C(=O)[N]=[N+]=[N-]')
    true_smiles = Chem.MolFromSmiles('C(N)N')
    true_smiles_2 = Chem.MolFromSmiles('C(=O)N')
    if molecule.HasSubstructMatch(primary_amine_pattern):
        modified_molecule = Chem.ReplaceSubstructs(molecule, primary_amine_pattern, diazonium_pattern, replaceAll=True)[0]
        if modified_molecule.HasSubstructMatch(wrong_smiles):
            modified_molecule = Chem.ReplaceSubstructs(modified_molecule, wrong_smiles, true_smiles, replaceAll=True)[0]
        elif modified_molecule.HasSubstructMatch(wrong_smiles_2):
            modified_molecule = Chem.ReplaceSubstructs(modified_molecule, wrong_smiles_2, true_smiles_2, replaceAll=True)[0]
        new_smiles = Chem.MolToSmiles(modified_molecule, isomericSmiles=True)
        return new_smiles
    elif '[NH2+]' in smiles:
        try:
            boc_protected_amine = Chem.MolFromSmiles(smiles)
            reaction = BocRemoval(bocamine=boc_protected_amine)
            molecule = reaction.get_product()
            modified_molecule = Chem.ReplaceSubstructs(molecule, primary_amine_pattern, diazonium_pattern, replaceAll=True)[0]
            new_smiles = Chem.MolToSmiles(modified_molecule, isomericSmiles=True)
            return new_smiles
        except:
            # print(smiles,"无法脱Boc")
            return None
    else:
        # print( smiles,"未找到一级胺")
        return None

In [55]:
def convert_diazo_to_triaz( acetylene_smiles,amine_smiles):
    rxn = Chem.ReactionFromSmarts('[*:6][C:1]#[C:2].[N-:3]=[N+:4]=[N:5]>>[N:5]1([C:2]=[C:1]([*:6])[N:3]=[N:4]1)')
    try:
        product = rxn.RunReactants((Chem.MolFromSmiles(amine_smiles), Chem.MolFromSmiles(acetylene_smiles)))
        # jb函数要两个括号
        a=Chem.MolToSmiles(product[0][0])
        if '[N+]=[N-]' in a or '[N-]=[N+]' in a:
            a = replace_specific_pattern(a,r'(?<!N=)\[N\+\]=\[N-\]|(?<!\[N\-\]=)\[N-\]=\[N\+\](?!\=N)','N=N')
        return a
    except:
        # print(amine_smiles,acetylene_smiles,"未找到反应产物")
        return None

In [56]:
def smiles_to_image_grid(smiles_list, img_size=(200, 200), mols_per_row=4, kekulize=True):
    mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]
    if kekulize:
        for mol in mols:
            Chem.Kekulize(mol)
    img = Draw.MolsToGridImage(mols, molsPerRow=mols_per_row, subImgSize=img_size)
    return img

In [57]:
def save_list_to_csv(filename, data_list):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        for item in data_list:
            csv_writer.writerow([item])

In [58]:
def is_supported_for_autodock(example_smiles):
    mol = Chem.MolFromSmiles(example_smiles)
    metal_atomic_numbers = [21,22,23,24,25,26,27,28,29,30,31,39,40,41,42,43,44,45,46,47,48,49,72,73,74,75,76,77,78,79,80,81]
    
    # AutoDock要求分子具有至少3个非氢原子
    if mol.GetNumHeavyAtoms() < 3:
        return False

    # 检查金属原子和原子类型
    for atom in mol.GetAtoms():
        atomic_num = atom.GetAtomicNum()

        # 检查金属原子
        if atomic_num in metal_atomic_numbers:
            return False

        # 检查原子类型是否为AutoDock支持的类型
        if atomic_num not in [1, 6, 7, 8, 9, 15, 16, 17, 35, 53]:
            return False

    return True

In [59]:
def filter_druglike(mol):
    """
    Filter molecules with druglike rule of 5 in RDKit.
    """
    if Lipinski.NumHDonors(mol) <= 5 and Lipinski.NumHAcceptors(mol) <= 10 and Descriptors.MolWt(mol) <= 500 and Crippen.MolLogP(mol)<=5 and Lipinski.NumRotatableBonds(mol)<=10:
        return True
    else:
        return False

In [60]:
with open ('/home/test/BiDe/BiDe_NH2.csv','r') as f:
    lines = f.readlines()
    amine_smiles = [l.split(',') for l in lines]
with open ('/home/test/BiDe/BiDe_term_acetylene.csv','r') as f:
    lines = f.readlines()
    ace_smiles = [l.split(',') for l in lines]
amine_smiles = amine_smiles[1:]
ace_smiles = ace_smiles[1:]

In [61]:
with open ('./N3_list_lower_65.csv','r') as f:
    N3_list = f.read().splitlines()
with open ('./ace_list_lower_65.csv','r') as f:
    ace_list = f.read().splitlines()

In [62]:
lower_65_NH2_dict = {}
lower_65_ace_dict = {}
for i in range(len(amine_smiles)):
    ligand_smiles = amine_smiles[i][-10].split('.')
    for j in range(len(ligand_smiles)):
        convert_result = convert_primary_amines_to_diazonium(ligand_smiles[j])
        if convert_result in N3_list:
            lower_65_NH2_dict[convert_result] = [amine_smiles[i][-10],amine_smiles[i][0],amine_smiles[i][-9]]

In [63]:
for i in range(len(ace_smiles)):
    if ace_smiles[i][-10] in ace_list:
        lower_65_ace_dict[ace_smiles[i][-10]] = [ace_smiles[i][-10],ace_smiles[i][0],ace_smiles[i][-9]]

In [72]:
def save_nested_dict_to_json(data, file_name):
    with open(file_name, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

In [73]:
save_nested_dict_to_json(lower_65_NH2_dict,'./lower_65_NH2_dict.json')
save_nested_dict_to_json(lower_65_ace_dict,'./lower_65_ace_dict.json')

In [80]:
Click_result_lower_65 = {}
k = 0
for i in tqdm(N3_list):
    for j in ace_list:
        b=convert_diazo_to_triaz(i,j)
        if filter_druglike(Chem.MolFromSmiles(b)) == True:
            Click_result_lower_65[f'{N3_list.index(i)}_{ace_list.index(j)}'] = b
        else :
            k += 1

100%|██████████| 6182/6182 [36:05<00:00,  2.86it/s]


In [81]:
save_nested_dict_to_json(Click_result_lower_65,'./Click_result_lower_65.json')